<a href="https://colab.research.google.com/github/LexKilpatrick/PhishNet-Combatant/blob/main/Phishing_Email_Detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Credit to DIMA806 on kaggle for a large portion of this code
# his notebook on phishing email detection can be found
# here: https://www.kaggle.com/code/dima806/phishing-email-detection-distilbert-huggingface/notebook

!pip install -q mlflow nlp
!pip install datasets
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 37.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 53.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.3/150.3 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.4/196.4 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.6/147.6 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.2/80.2 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

In [ ]:
import pandas as pd
import gc
import re
import numpy as np
import torch
from transformers import AutoModel, AutoTokenizer
from transformers import TextDataset, LineByLineTextDataset, DataCollatorForLanguageModeling, \
pipeline, Trainer, TrainingArguments, DataCollatorWithPadding
from transformers import AutoModelForSequenceClassification
from nlp import Dataset
from imblearn.over_sampling import RandomOverSampler
from transformers import pipeline
import datasets
from bs4 import BeautifulSoup
from tqdm import tqdm
tqdm.pandas()

In [ ]:

train_fraction = 0.8
num_train_epochs = 4
batch_size = 16
warmup_steps = 50
weight_decay = 0.02
BERT_MODEL = "distilbert-base-cased"
output_dir = "./phishing-email-detection"

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Phishing-Email-Datasets/Phishing_Email.csv").drop(['Unnamed: 0'], axis=1).drop_duplicates()
print(df.shape)
df.sample(5).T

(17539, 2)


,10044,10832,13395,16271,6398
Email Text,Marc Perkel writes:> Here something I want - ...,get big money at this casino site click here t...,comutational linguistics - general & thematic ...,This mixer is specific to ALSA's special stuff...,"Dear zzzz ,\nÂ Â Â Â Â Â Â Â Â Â Want \n ..."
Email Type,Safe Email,Phishing Email,Safe Email,Safe Email,Phishing Email


In [ ]:
df['label'] = (df['Email Type']=="Phishing Email").astype(int)
df['title'] = df['Email Text']
df = df[['title', 'label']]

print(df.shape)
df.sample(20)

(17539, 2)


,title,label
8638,view and print doorstep reports view reports s...,0
12760,brooklyn navy yard legal risk memo attached is...,0
7146,"On Tue, Jul 30, 2002 at 03:47:40PM +0100 or so...",0
2828,"oral surgery vince , anita and kevin : i went ...",0
202,"\n >> Before we get too far down this road,...",0
15741,free computer - find out how to get it !,1
17770,use Perl Daily NewsletterIn this issue:\n *...,0
1051,"On Thu, Aug 08, 2002 at 11:27:59AM +0100, wint...",0
3030,re : interim master terms agreement attached h...,0
9929,Today's Headlines from The Register\n---------...,0


In [ ]:
df.dropna(inplace=True)


In [ ]:
df['label'].mean()


0.37393089291823467

In [ ]:
y = df[['label']]
df = df.drop(['label'], axis=1)
ros = RandomOverSampler(random_state=83)
df, y_resampled = ros.fit_resample(df, y)
del y
df['label'] = y_resampled
print(df.shape)
del y_resampled
gc.collect()

(21960, 2)


123

In [ ]:
medium_dataset = Dataset.from_pandas(df)


In [ ]:
del df
gc.collect()

0

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(BERT_MODEL, use_fast=True, low_cpu_mem_usage=False)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/465 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples["title"], truncation=True)

medium_dataset = medium_dataset.map(preprocess_function, batched=True)

  0%|          | 0/22 [00:00<?, ?it/s]

In [ ]:
medium_dataset = medium_dataset.train_test_split(test_size=1-train_fraction)


  0%|          | 0/18 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
medium_dataset.remove_column_("title")

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


In [ ]:
tokenizer.decode(medium_dataset['train'][0]['input_ids'])


"[CLS] ccall / celao 3 dear colleague, the following is the first call for papers for the ccall 3 conference, which will be held next june. please forward it to any other colleagues you believe might be interested. thank you. cher ( e ) collegue, voici le premier appel de communications pour le colloque celao 3, qui aura lieu au mois de juin prochain. veuillez avoir l'obligeance de le faire parvenir a d'autres collegues qui pourraient s'y interesser. merci d'avance. ian m. richmond. - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - ccall 3 / celao 3 # # # # # # # # # # # # # # # # # troisieme congres canadien sur l'enseignement des langues assiste par ordinateur third canadian conference on computer - assisted language learning 25 - 27 june, 1998 / 25 - 27 juin 1998 universite sainte - anne pointe - de - l'eglise, n. - e. / church point, ns call for papers the third canadian conference on computer - assisted language learning will be he

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(BERT_MODEL, num_labels=2,
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False # Whether the model returns all hidden-states.
)

model.config.id2label = {0: 'SAVE EMAIL', 1: 'PHISHING EMAIL'}

model.safetensors:   0%|          | 0.00/263M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
print(model.num_parameters(only_trainable=True)/1e6)


65.783042


In [ ]:
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

<ipython-input-20-e1ee7db0da93>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [ ]:
training_args = TrainingArguments(
    output_dir=output_dir,
    logging_dir='./logs',
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    logging_strategy='steps',
    logging_first_step=True,
    load_best_model_at_end=True,
    logging_steps=1,
    evaluation_strategy='epoch',
    warmup_steps=warmup_steps,
    weight_decay=weight_decay,
    eval_steps=1,
    save_strategy='epoch',
    report_to="mlflow",  # log to mlflow
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=medium_dataset['train'],
    eval_dataset=medium_dataset['test'],
    data_collator=data_collator
)

In [ ]:
trainer.evaluate()


You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'eval_loss': 0.6936135292053223,
 'eval_accuracy': 0.48975409836065575,
 'eval_runtime': 69.2134,
 'eval_samples_per_second': 63.456,
 'eval_steps_per_second': 3.973}

In [ ]:
trainer.train()


Epoch,Training Loss,Validation Loss,Accuracy
1,0.000800,0.035300,0.991120
2,0.000300,0.031871,0.992259
3,0.000000,0.037565,0.994763
4,0.000000,0.036024,0.994991


TrainOutput(global_step=4392, training_loss=0.03549300747936253, metrics={'train_runtime': 3655.6623, 'train_samples_per_second': 19.223, 'train_steps_per_second': 1.201, 'total_flos': 9298925752822848.0, 'train_loss': 0.03549300747936253, 'epoch': 4.0})

In [ ]:
trainer.evaluate()


{'eval_loss': 0.03187074139714241,
 'eval_accuracy': 0.9922586520947176,
 'eval_runtime': 75.1568,
 'eval_samples_per_second': 58.438,
 'eval_steps_per_second': 3.659,
 'epoch': 4.0}

In [ ]:
trainer.save_model()


In [ ]:
tokenizer.save_vocabulary(save_directory=output_dir)


('./phishing-email-detection/vocab.txt',)

In [ ]:
pipe = pipeline("text-classification", output_dir, tokenizer=BERT_MODEL)
sample_title = '''Why do employees leave companies — analysis of IBM employee data'''
pipe(sample_title, top_k=None)

[{'label': 'SAVE EMAIL', 'score': 0.9998961687088013},
 {'label': 'PHISHING EMAIL', 'score': 0.00010386609937995672}]

In [ ]:
sample_title2 = sample_title.upper()
pipe(sample_title2, top_k=None)

[{'label': 'PHISHING EMAIL', 'score': 0.998894989490509},
 {'label': 'SAVE EMAIL', 'score': 0.001105019124224782}]

In [ ]:
sample_title3 = '''Why do employees leave companies?'''
pipe(sample_title3, top_k=None)

[{'label': 'SAVE EMAIL', 'score': 0.9976263642311096},
 {'label': 'PHISHING EMAIL', 'score': 0.0023736595176160336}]